In [ ]:
-- Wypisz wszystkich członków biblioteki z adresami i info czy jest dzieckiem czy nie i
-- ilość wypożyczeń w poszczególnych latach i miesiącach.
USE library;

SELECT adult.member_no,adult.street,adult.city,adult.state, YEAR(loan.out_date), MONTH(loan.out_date), COUNT(loan.out_date), 'adult'
FROM adult 
JOIN loan ON loan.member_no=adult.member_no
GROUP BY adult.member_no,adult.street,adult.city,adult.state, YEAR(loan.out_date), MONTH(loan.out_date)
UNION
SELECT juvenile.member_no,adult.street,adult.city,adult.state, YEAR(loan.out_date), MONTH(loan.out_date), COUNT(loan.out_date), 'child'
FROM juvenile
JOIN adult ON juvenile.adult_member_no=adult.member_no
JOIN loan ON loan.member_no=adult.member_no
GROUP BY juvenile.member_no,adult.street,adult.city,adult.state, YEAR(loan.out_date), MONTH(loan.out_date)


In [ ]:
-- Zamówienia z Freight większym niż AVG danego roku.
USE Northwind;
SELECT o.OrderID, o.Freight
FROM Orders o
WHERE o.Freight > (SELECT AVG(Freight)
                FROM Orders 
                WHERE YEAR(OrderDate)=YEAR(o.OrderDate))


In [ ]:
-- Klienci, którzy nie zamówili nigdy nic z kategorii 'Seafood' 
USE Northwind;
SELECT DISTINCT CustomerID 
FROM Orders
WHERE CustomerID IN (SELECT o.CustomerID
                    FROM Orders o
                    JOIN [Order Details] od ON o.OrderID=od.OrderID
                    JOIN Products p ON od.ProductID=p.ProductID
                    JOIN Categories c ON p.CategoryID=c.CategoryID
                    WHERE c.CategoryName LIKE 'Seafood')


In [ ]:
-- Dla każdego klienta najczęściej zamawianą kategorię (liczba sztuk)

SELECT oo.CustomerID, pp.CategoryID, SUM(odd.Quantity)
FROM Orders oo
JOIN [Order Details] odd ON oo.OrderID=odd.OrderID
JOIN Products pp ON odd.ProductID=pp.ProductID
GROUP BY oo.CustomerID, pp.CategoryID
HAVING SUM(odd.Quantity)=(SELECT MAX(q)
        FROM (SELECT o.CustomerID, p.CategoryID, SUM(od.Quantity) q
                FROM Orders o
                JOIN [Order Details] od ON o.OrderID=od.OrderID
                JOIN Products p ON od.ProductID=p.ProductID
                GROUP BY o.CustomerID, p.CategoryID) tmp
        WHERE tmp.CustomerID=oo.CustomerID)


In [ ]:
-- Podział na company, year month i suma freight 
USE Northwind;

SELECT CustomerID, YEAR(OrderDate), MONTH(OrderDate), SUM(Freight)
FROM Orders
GROUP BY CustomerID, YEAR(OrderDate), MONTH(OrderDate)

In [ ]:
-- Wypisać wszystkich czytelników, którzy nigdy nie wypożyczyli książki dane
-- adresowe i podział czy ta osoba jest dzieckiem
USE library;
SELECT DISTINCT member_no, street, city, zip, 'adult'
FROM adult
WHERE member_no NOT IN (SELECT DISTINCT member_no 
FROM loan)
UNION 
SELECT DISTINCT j.member_no, a.street, a.city, a.zip, 'child'
FROM juvenile j
JOIN adult a ON j.adult_member_no=a.member_no 
WHERE j.member_no NOT IN (SELECT DISTINCT member_no 
FROM loan)

In [ ]:
-- Najczęściej wybierana kategoria w 1997 dla każdego klienta
USE Northwind;
SELECT Orders.CustomerID, Products.CategoryID
FROM Orders 
JOIN [Order Details] ON [Order Details].OrderID=Orders.OrderID
JOIN Products ON [Order Details].ProductID=Products.ProductID 
WHERE YEAR(Orders.OrderDate)=1997
GROUP BY Orders.CustomerID, Products.CategoryID 
HAVING  COUNT(Orders.OrderID) = (SELECT MAX(cnt)
                                  FROM (SELECT o.CustomerID cusid, p.CategoryID catid, COUNT(*) cnt
                                        FROM Orders o 
                                        JOIN [Order Details] od ON o.OrderID=od.OrderID
                                        JOIN Products p ON od.ProductID=p.ProductID 
                                        WHERE YEAR(o.OrderDate)=1997 
                                        GROUP BY o.CustomerID, p.CategoryID) tmp
                                   WHERE tmp.cusid=Orders.CustomerID)


In [ ]:
-- Dla każdego czytelnika imię nazwisko, suma książek wypożyczony przez tą osobę i
-- jej dzieci, który żyje w Arizona ma mieć więcej niż 2 dzieci lub kto żyje w Kalifornii
-- ma mieć więcej niż 3 dzieci
USE library;
SELECT a.member_no, m.firstname, m.lastname, COUNT(l.member_no)+ --nie chce mi sie tego dopisywać naprawde zeby tez dzieci dodawac
FROM adult a 
LEFT OUTER JOIN member m ON a.member_no=m.member_no
LEFT OUTER JOIN loan l ON a.member_no=l.member_no
WHERE (a.state='AZ' AND (SELECT COUNT(adult_member_no)
        FROM juvenile
        WHERE a.member_no=adult_member_no)>2 )OR (a.state='CA' AND (SELECT COUNT(adult_member_no)
        FROM juvenile
        WHERE a.member_no=adult_member_no)>3)
GROUP BY a.member_no, m.firstname, m.lastname
